In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import glob

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)

In [17]:
archivos = glob.glob("../data/raw/5.0-poblacion-fonasa/*.csv")
df_fonasa = pd.concat(pd.read_csv(archivo, encoding="latin-1") for archivo in archivos)

df_fonasa["ANO_INFORMACION"] = df_fonasa.MES_INFORMACION.astype(str).str[:4]

df_fonasa["REGION"] = df_fonasa["REGION"].str.upper().str.strip()
df_fonasa["SEXO"] = df_fonasa["SEXO"].str.upper().str.strip()
df_fonasa["SERVICIO_SALUD"] = df_fonasa["SERVICIO_SALUD"].str.upper().str.strip()
df_fonasa["COMUNA"] = df_fonasa["COMUNA"].str.upper().str.strip()

df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].replace(
    {"Más de 99 años": "99 años", "S.I.": "-1"}
)
df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].str.split().str[0].astype(int)

In [5]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in df_pais_fonasa["REGION"].unique():
    df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
    ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO ORIENTE'")
df_ssmc_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO CENTRAL'")

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa

In [9]:
ESTRATOS_A_CALCULAR_FONASA["Pais"]["EDAD_TRAMO"].value_counts()

EDAD_TRAMO
20 a 24 años      138712
25 a 29 años      132628
60 a 64 años      127505
30 a 34 años      123612
35 a 39 años      121486
50 a 54 años      120456
45 a 49 años      120429
40 a 44 años      119824
65 a 69 años      119401
55 a 59 años      119332
15 a 19 años      103927
70 a 74 años       82629
10 a 14 años       66964
05 a 09 años       65544
75 a 79 años       60514
03 a 04 años       51847
00 a 02 años       50573
80 a 84 años       43702
85 a 89 años       31711
90 a 94 años       21490
95 a 99 años       12796
Más de 99 años      8258
S.I.                5127
Name: count, dtype: int64